In [1]:
import sqlhelpers as sh

In [2]:
# %load Exercise Files\\Chapters\\sql-chap09.txt
-- 02 transactions
-- test.db

CREATE TABLE widgetInventory (
  id INTEGER PRIMARY KEY,
  description TEXT,
  onhand INTEGER NOT NULL
);

CREATE TABLE widgetSales (
  id INTEGER PRIMARY KEY,
  inv_id INTEGER,
  quan INTEGER,
  price INTEGER
);

INSERT INTO widgetInventory ( description, onhand ) VALUES ( 'rock', 25 );
INSERT INTO widgetInventory ( description, onhand ) VALUES ( 'paper', 25 );
INSERT INTO widgetInventory ( description, onhand ) VALUES ( 'scissors', 25 );

SELECT * FROM widgetInventory;

BEGIN TRANSACTION;
INSERT INTO widgetSales ( inv_id, quan, price ) VALUES ( 1, 5, 500 );
UPDATE widgetInventory SET onhand = ( onhand - 5 ) WHERE id = 1;
END TRANSACTION;

SELECT * FROM widgetInventory;
SELECT * FROM widgetSales;

BEGIN TRANSACTION;
INSERT INTO widgetInventory ( description, onhand ) VALUES ( 'toy', 25 );
ROLLBACK;
SELECT * FROM widgetInventory;

-- restore database
DROP TABLE widgetInventory;
DROP TABLE widgetSales;

-- 03 performance
-- test.db

CREATE TABLE test ( id INTEGER PRIMARY KEY, data TEXT );

-- copy / paste 1,000 of these ...
INSERT INTO test ( data ) VALUES ( 'this is a good sized line of text.' );

-- put this before the 1,000 INSERT statements
BEGIN TRANSACTION;

-- put this after the 1,000 INSERT statements
END TRANSACTION;

SELECT COUNT(*) FROM test;

-- restore database
DROP TABLE test;



# Data Integrity

In [3]:
q = """
CREATE TABLE widgetInventory (
  id INTEGER PRIMARY KEY,
  description TEXT,
  onhand INTEGER NOT NULL
);
"""
sh.perform_insert_and_stats(q, 'test.db')

### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| 
CREATE TABLE widgetInventory (
  id INTEGER PRIMARY KEY,
  description TEXT,
  onhand INTEGER NOT NULL
);
 |||


In [4]:
q = """
CREATE TABLE widgetSales (
  id INTEGER PRIMARY KEY,
  inv_id INTEGER,
  quan INTEGER,
  price INTEGER
);
"""
sh.perform_insert_and_stats(q, 'test.db')

### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| 
CREATE TABLE widgetSales (
  id INTEGER PRIMARY KEY,
  inv_id INTEGER,
  quan INTEGER,
  price INTEGER
);
 |||


In [6]:

q1 = "INSERT INTO widgetInventory ( description, onhand ) VALUES ( 'rock', 25 );"
q2 = "INSERT INTO widgetInventory ( description, onhand ) VALUES ( 'paper', 25 );"
q3 = "INSERT INTO widgetInventory ( description, onhand ) VALUES ( 'scissors', 25 );"

sh.perform_insert_and_stats(q1, 'test.db')
sh.perform_insert_and_stats(q2, 'test.db')
sh.perform_insert_and_stats(q3, 'test.db')

### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| INSERT INTO widgetInventory ( description, onhand ) VALUES ( 'rock', 25 ); |||
### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| INSERT INTO widgetInventory ( description, onhand ) VALUES ( 'paper', 25 ); |||
### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| INSERT INTO widgetInventory ( description, onhand ) VALUES ( 'scissors', 25 ); |||


In [7]:

q = "SELECT * FROM widgetInventory;"
sh.query_result_stats(q, "test.db")

### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| SELECT * FROM widgetInventory; |||

There are 3 rows in the query results table
There are 3 columns in the query results table

First 5 rows of the query results:
   id description  onhand
0   1        rock      25
1   2       paper      25
2   3    scissors      25


In [9]:
# lets do a transaction

q1 = "BEGIN TRANSACTION;"
q2 = "INSERT INTO widgetSales ( inv_id, quan, price ) VALUES ( 1, 5, 500 );"
q3 = "UPDATE widgetInventory SET onhand = ( onhand - 5 ) WHERE id = 1;"
q4 = "END TRANSACTION;"


sh.perform_insert_and_stats(q1, "test.db")
sh.perform_insert_and_stats(q2, "test.db")
sh.perform_insert_and_stats(q3, "test.db")
sh.perform_insert_and_stats(q4, "test.db")

### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| BEGIN TRANSACTION; |||
### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| INSERT INTO widgetSales ( inv_id, quan, price ) VALUES ( 1, 5, 500 ); |||
### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| UPDATE widgetInventory SET onhand = ( onhand - 5 ) WHERE id = 1; |||


OperationalError: cannot commit - no transaction is active

In [11]:
q1 = "DROP TABLE widgetInventory;"
q2 = "DROP TABLE widgetSales;"

sh.perform_insert_and_stats(q1, "test.db")
sh.perform_insert_and_stats(q2, "test.db")

### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| DROP TABLE widgetInventory; |||
### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| DROP TABLE widgetSales; |||


# Performance